In [1]:
FILE_NAME='data/docx/sample-docx.docx'

Approaches to read the docx file in Python:
1. Convert to HTML with mammoth.js and then use BeautifulSoup to parse the HTML
2. Use unstructered.io  to extract the text from the docx file

# mammoth.js


In [ ]:
%pip install mammoth

In [ ]:
import mammoth

with open(FILE_NAME, "rb") as docx_file:
    result = mammoth.convert_to_html(docx_file,)
    html = result.value 
    messages = result.messages 
    
with open(FILE_NAME.replace(".docx",".html"), "w", encoding="utf-8") as html_file:
    html_file.write(html)

# MarkItDown 
MarkItDown is a lightweight Python utility for converting various files to Markdown for use with LLMs and related text analysis pipelines. To this end, it is most comparable to [textract](https://github.com/deanmalmgren/textract), but with a focus on preserving important document structure and content as Markdown (including: headings, lists, tables, links, etc.) While the output is often reasonably presentable and human-friendly, it is meant to be consumed by text analysis tools -- and may not be the best option for high-fidelity document conversions for human consumption.

At present, MarkItDown supports:

- PDF
- PowerPoint
- Word
- Excel
- Images (EXIF metadata and OCR)
- Audio (EXIF metadata and speech transcription)
- HTML
- Text-based formats (CSV, JSON, XML)
- ZIP files (iterates over contents)
- Youtube URLs
- EPubs
- ... and more! 

In [2]:
from markitdown import MarkItDown

md = MarkItDown() # Set to True to enable plugins
result = md.convert(FILE_NAME)
print(result.text_content)

Exploring Azure AI Solutions

Empowering Businesses with Intelligent Capabilities

# Introduction

In the rapidly evolving landscape of artificial intelligence, Azure AI solutions stand at the forefront, offering a comprehensive suite of tools and services designed to help businesses harness the power of AI. From natural language processing to computer vision, Azure AI provides a robust and scalable platform for integrating intelligent capabilities into diverse applications.

# Key Azure AI Services

## Azure Cognitive Services

Azure Cognitive Services offer a range of pre-built AI models that can be easily integrated into applications to enhance their functionality. These services include:

* Language Understanding (LUIS): Enables applications to understand and process natural language input.
* Text Analytics: Provides sentiment analysis, key phrase extraction, and language detection capabilities.
* Computer Vision: Analyzes images and videos to extract information and insights, incl

# unstructered.io

In [ ]:
%pip  install "unstructured[docx]"

In [ ]:
from unstructured.partition.docx import partition_docx
elements = partition_docx(FILE_NAME)
print(elements)

In [ ]:
print("Number of elements: ", len(elements))
for i, element in enumerate(elements):             
    if element.category == 'Table':
        chunk_text = element.metadata.text_as_html
    else:
        if element.category == 'Title':
            chunk_text = "# "+ element.text
        else:
            chunk_text = element.text 
    print(f'element {i} ({element.category}): Chunk len ({len(chunk_text)}) {chunk_text[:100]}...') 
    

In [ ]:
MAX_CHARACTERS = 1500
NEW_AFTER_N_CHARS = 2000
COMBINE_UNDER_N_CHARS = 100

from unstructured.chunking.title import chunk_by_title
from unstructured.cleaners.core import clean

chunks = chunk_by_title(elements, multipage_sections=True, max_characters=MAX_CHARACTERS, new_after_n_chars=NEW_AFTER_N_CHARS, combine_text_under_n_chars=COMBINE_UNDER_N_CHARS)  

out_text = ''
chunks_list=[]
# Complete and write chunks
for i, chunk in enumerate(chunks): 
            
    if chunk.category == 'Table':
        chunk_text = chunk.metadata.text_as_html
    else:
        chunk_text = chunk.text        
    chunks_list.append(chunk_text)
    print(f'Chunk {i} ({chunk.category}): Chunk len ({len(chunk_text)}) {chunk_text[:100]}...') 
    out_text += chunk_text
                      
cleaned_text = clean(out_text, extra_whitespace=True)
print(cleaned_text)
print(f'Number of chunks: {len(chunks)}')